In [148]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt

#for replicability purposes
tf.random.set_seed(91195003)
#for an easy reset backend session state
tf.keras.backend.clear_session()

In [205]:
n_variate = 1

In [206]:

#Load dataset
def load_dataset(path=r'data_covid.csv'):
    return pd.read_csv(path)

In [207]:
#split data into training and validation sets
def split_data(training, perc=10):
    train_idx = np.arange(0, int(len(training)*(100-perc)/100))
    val_idx = np.arange(int(len(training)*(100-perc)/100+1), len(training))
    return train_idx, val_idx

In [208]:
def prepare_data(df):
    df_aux = df
    df_aux["Data"] = pd.to_datetime(df_aux["Data"])
    df_aux = df_aux.set_index('Data')
    # por obitos como primeira coluna
    df_aux = df_aux.reindex(columns=['confirmados_arsnorte', 'confirmados_arscentro', 'confirmados_arslvt',
       'confirmados_arsalentejo', 'confirmados_arsalgarve',
       'confirmados_acores', 'confirmados_madeira', 'confirmados_novos',
       'recuperados', 'obitos', 'internados', 'internados_uci',
       'confirmados_0_9_f', 'confirmados_0_9_m', 'confirmados_10_19_f',
       'confirmados_10_19_m', 'confirmados_20_29_f', 'confirmados_20_29_m',
       'confirmados_30_39_f', 'confirmados_30_39_m', 'confirmados_40_49_f',
       'confirmados_40_49_m', 'confirmados_50_59_f', 'confirmados_50_59_m',
       'confirmados_60_69_f', 'confirmados_60_69_m', 'confirmados_70_79_f',
       'confirmados_70_79_m', 'confirmados_80_plus_f', 'confirmados_80_plus_m',
       'obitos_arsnorte', 'obitos_arscentro', 'obitos_arslvt',
       'obitos_arsalentejo', 'obitos_arsalgarve', 'obitos_acores',
       'obitos_madeira', 'obitos_0_9_f', 'obitos_0_9_m', 'obitos_10_19_f',
       'obitos_10_19_m', 'obitos_20_29_f', 'obitos_20_29_m', 'obitos_30_39_f',
       'obitos_30_39_m', 'obitos_40_49_f', 'obitos_40_49_m', 'obitos_50_59_f',
       'obitos_50_59_m', 'obitos_60_69_f', 'obitos_60_69_m', 'obitos_70_79_f',
       'obitos_70_79_m', 'obitos_80_plus_f', 'obitos_80_plus_m', 'ativos',
       'internados_enfermaria', 'Max_Temp', 'Min_Temp', 'Temperature',
       'Precipitation', 'Wind_Speed', 'Wind_Direction', 'Visibility',
       'Cloud_Cover', 'Relative_Humidity', 'Rain', 'Clear',
       'Partially_cloudy'])
    # drop de features com correlação alta a outras
    df_aux = df_aux.drop(df_aux.columns.difference(['obitos','confirmados_arsnorte' ,'confirmados_arscentro', 'confirmados_arslvt', 'confirmados_arsalgarve', 'confirmados_acores', 'confirmados_madeira', 'confirmados_novos', 'recuperados', 'internados', 'obitos_arsnorte', 'obitos_arscentro', 'obitos_arsalentejo', 'obitos_80_plus_m', 'Wind_Direction', 'Visibility', 'obitos_0_9_m', 'Cloud_Cover', 'Partially_cloudy']), axis =1)

    # df_aux = df_aux.drop(columns=[ 'obitos_0_9_f', 'obitos_10_19_f', 'obitos_10_19_m', 'obitos_20_29_f', 'obitos_20_29_m',
    #                               'ativos', 'Partially_cloudy', 'commercial_flights'])
    return df_aux

In [209]:
# def data_normalization(df, norm_range=(-1, 1)):
#   #[-1, 1] for LSTM due to the internal use of tanh by the memory cell
#   scaler = MinMaxScaler(feature_range=norm_range)
#   df = scaler.fit_transform(df)
#   return scaler

In [210]:
def data_normalization(df, norm_range=(-1, 1)):
    #[-1, 1] for LSTM due to the internal use of tanh by the memory cell
    scaler = MinMaxScaler(feature_range=norm_range)
    colunas = [x for x in df.columns if x != 'obitos']
    for col in colunas:
        df[[col]] = scaler.fit_transform(df[[col]])
    df[['obitos']] = scaler.fit_transform(df[['obitos']])
    return scaler

In [211]:
#plot learning curve
def plot_learning_curves(history, epochs):
    #accuracies and losses
    #dict_keys(['loss', 'mae', 'rmse', 'val_loss', 'val_mae', 'val_rmse'])
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    mae=history.history['mae']
    val_mae=history.history['val_mae']
    rmse=history.history['rmse']
    val_rmse=history.history['val_rmse']
    epochs_range = range(epochs)
    #creating figure
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,2)
    plt.plot(epochs_range,loss,label='Training Loss')
    plt.plot(epochs_range,val_loss,label='Validation Loss')
    plt.plot(epochs_range,mae,label='Training MAE')
    plt.plot(epochs_range,val_mae,label='Validation MAE')
    plt.plot(epochs_range,rmse,label='Training RMSE')
    plt.plot(epochs_range,val_rmse,label='Validation RMSE')
    plt.legend(loc='upper right')
    plt.title('Training/Validation Loss')
    plt.show()

In [212]:
#Plot time series data
def plot_confirmed_cases(data):
    plt.figure(figsize=(8,6))
    plt.plot(range(len(data)), data)
    plt.title('Confirmed Cases of COVID-19')
    plt.ylabel('Cases')
    plt.xlabel('Days')
    plt.show()

In [213]:
#Preparing the dataset for the LSTM
def to_supervised(df, timesteps):
    data = df.values
    X, y = list(), list()
    #iterate over the training set to create X and y
    dataset_size = len(data)
    for curr_pos in range(dataset_size):
        #end of the input sequence is the current position + the number of timesteps of the input sequence
        input_index = curr_pos + timesteps
        #end of the labels corresponds to the end of the input sequence + 1
        label_index = input_index + 1
        #if we have enough data for this sequence
        if label_index < dataset_size:
            X.append(data[curr_pos:input_index, :])
            y.append(data[input_index:label_index, 0])
            # y.append(data[input_index:label_index, 0:n_variate])

    #using np.float32 for GPU performance
    return np.array(X).astype('float32'), np.array(y).astype('float32')

In [214]:
#Building the model
def rmse(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))


In [215]:
def build_model(timesteps, features, h_neurons=64, activation='tanh'):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(h_neurons, activation=activation, input_shape=(timesteps, features), return_sequences=True))
    #Add a new layer
    model.add(tf.keras.layers.LSTM(32, activation=activation ,return_sequences=False))
    #
    model.add(tf.keras.layers.Dense(h_neurons, activation=activation))
    # model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    #model summary (and save it as PNG)
    tf.keras.utils.plot_model(model, 'Covid_model.png', show_shapes=True)
    return model

In [216]:
#Compiling and fit the model
def compile_and_fit(model, epochs, batch_size):
    #compile
    model.compile(loss = rmse, optimizer = tf.keras.optimizers.Adam(), metrics = ['mae', rmse])
    #fit
    hist_list = list()
    loss_list = list()

    #callback
    #saving in Keras HDF5 (or h5), a binary data format
    callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath='my_model_{epoch}_{val_loss:.3f}.h5',#path where to save model
        save_best_only=True,#overwrite the current checkpoint if and only if
        monitor='val_loss',#the val_loss score has improved
        save_weights_only=False,#if True, only the weigths are saved
        verbose=1,#verbosity mode
        period=5 #save ony at the fifth epoch (5 em 5 epocas) 
        )#,
    #interrupt training if loss stops improving for over 2 epochs
    #tf.keras.callbacks.EarlyStopping(patience=9, monitor='cost')
    ]

    #Time Series Cross Validator
    tscv = TimeSeriesSplit(n_splits=cv_splits)
    for train_index, test_index in tscv.split(X):
        train_idx, val_idx = split_data(train_index, perc=10) #further split into training and validation sets
        #build data
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]
        X_test, y_test = X[test_index], y[test_index]

        #print("x_val::::",X_val,"\n","y_val:",y_val,"\n")

        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, shuffle=False)
        # history = model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, shuffle=False, callbacks=callbacks)
        metrics = model.evaluate(X_test, y_test)


        plot_learning_curves(history, epochs)
        hist_list.append(history)


    return model, hist_list

In [217]:
#Main Execution
#the dataframes
df_raw = load_dataset()
df_data = prepare_data(df_raw)
df = df_data.copy()
n_variate = len(df.columns)

scaler = data_normalization(df) #scaling data to [-1, 1]

In [218]:
df

,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,confirmados_novos,recuperados,obitos,internados,obitos_arsnorte,obitos_arscentro,obitos_arsalentejo,obitos_0_9_m,obitos_80_plus_m,Wind_Direction,Visibility,Cloud_Cover,Partially_cloudy
Data,,,,,,,,,,,,,,,,,,,
2020-02-29,0.202582,0.121951,0.384871,0.041451,-0.137255,0.073171,-1.000000,0.158051,0.071575,-1.000000,-1.000000,-1.000000,-1.000000,0.0,0.498163,-0.366613,-0.410911,-0.499570,-1.000000
2020-03-31,0.525819,0.232578,0.452042,0.150259,0.137255,0.105691,-0.951486,0.158051,0.153374,-0.933539,-0.872992,-0.933610,-1.000000,0.0,0.512862,-0.139100,0.175586,-0.723218,-1.000000
2020-04-30,-0.025323,0.054007,0.354009,-0.062176,-0.333333,0.040650,-0.887569,0.167756,0.055215,-0.549284,-0.260903,-0.737759,-0.995546,0.0,0.568132,0.133246,-0.665458,-0.721414,-1.000000
2020-05-31,0.117676,0.083624,0.429652,0.051813,-0.215686,0.073171,-0.949107,0.192513,0.063395,-0.717575,-0.666412,-0.933610,-1.000000,0.0,0.533147,0.154064,-0.164087,-0.050266,0.806452
2020-06-30,0.202085,0.126307,0.369743,0.041451,-0.137255,0.089431,-0.936963,0.173302,0.047035,-0.826230,-0.947972,-0.983402,-0.973274,0.0,0.513156,1.000000,0.742179,-0.392718,-1.000000
2020-07-31,0.216981,0.116725,0.357640,0.051813,-0.098039,0.056911,-0.941982,0.166568,0.071575,-0.809608,-0.984698,-0.993361,-0.937639,0.0,0.511686,0.611871,1.000000,-0.923575,-1.000000
2020-08-31,0.219464,0.124564,0.387595,0.051813,-0.058824,0.073171,-0.954764,0.105169,0.051125,-0.855396,-0.967865,-0.998340,-0.995546,0.0,0.506982,0.993399,0.982309,-0.512946,-0.638710
2020-09-30,0.307349,0.187282,0.463843,0.217617,-0.058824,0.105691,-0.885738,0.209745,0.092025,-0.806217,-0.943382,-0.983402,-0.995546,0.0,0.511098,-0.013566,0.625181,-0.530173,-1.000000
2020-10-31,1.000000,0.531359,0.689864,0.243523,-0.215686,0.121951,-0.571520,0.652010,0.198364,-0.522285,-0.655700,-0.915353,-0.893096,0.0,0.549316,0.091149,-0.188854,-0.580283,-1.000000


In [219]:
#Recursive Multi-Step Forecast!!!
def forecast(model, df, timesteps, multisteps, scaler):
    input_seq = np.array(df[-timesteps:].values) #getting the last sequence of known value
    inp = input_seq
    #print("Input_seq: ",inp)
    forecasts = list()

    #multisteps tells us how many iterations we want to perform, i.e., how many days we want to predict
    for step in range(1, multisteps+1):
        inp = inp.reshape(1,timesteps,n_variate)
        yhat = model.predict(inp) #dá o valor predito normalizado
        yhat_desnormalized = scaler.inverse_transform(yhat) #dá valor predito desnormalizado
        forecasts.append(yhat_desnormalized) #adicionar previsao à lista final de previsões
        # se for necessário prever mais do que 1 semana
        # list_yhat = [yhat[0][i] for i in range(len(yhat[0]))]
        # print('list_yhat:')
        # print(list_yhat)
        # #preparar novo input para fazer previsão para a semana seguinte
        # inp= np.append(inp[0],[list_yhat],axis=0) #adiciona previsão recente ao input
        # inp = inp[-timesteps:] #vai ao input buscar os ultimos timesteps registados
        # print('forecasts:')
        # print(forecasts)
    return forecasts



In [220]:
def plot_forecast(data, forecasts):
    plt.figure(figsize=(20,8))
    plt.plot(range(len(data)), data['obitos'], color='green', label='True value')
    plt.plot(range(len(data)-1, len(data)+len(forecasts)-1), forecasts, color='red', label='Forecasts')
    plt.title('Óbitos em Portugal')
    plt.ylabel('Número de Óbitos')
    plt.xlabel('Meses')
    plt.legend()
    plt.show()

# Tunning

In [221]:
tunning_dict = {               
                1: {'timesteps' : 1, 'multisteps' : 1, 'cv_splits': 2, 'epochs' : 25,  'batch_size' : 2 }#,
                # 2: {'timesteps' : 4, 'multisteps' : 4, 'cv_splits' : 10, 'epochs' : 50,  'batch_size' : 2 },
                # 3: {'timesteps' : 4, 'multisteps' : 4, 'cv_splits' : 10, 'epochs' : 100,  'batch_size' : 2 },

                #
                # 4: {'timesteps' : 14, 'multisteps' : 100, 'cv_splits': 10, 'epochs' : 10,  'batch_size' : 2 },
                # 5: {'timesteps' : 14, 'multisteps' : 100, 'cv_splits' : 10, 'epochs' : 50,  'batch_size' : 2 },
                # 6: {'timesteps' : 14, 'multisteps' : 100, 'cv_splits' : 10, 'epochs' : 100,  'batch_size' : 2 }#,
                #
                #7: {'timesteps' : 30, 'multisteps' : 100, 'cv_splits': 5, 'epochs' : 10,  'batch_size' : 1 },
                #8: {'timesteps' : 30, 'multisteps' : 100, 'cv_splits' : 5, 'epochs' : 50,  'batch_size' : 1 },
                #9: {'timesteps' : 30, 'multisteps' : 100, 'cv_splits' : 5, 'epochs' : 100,  'batch_size' : 1 }

                }
# record da history de cada modelo
record = {}

In [222]:
for t in tunning_dict:
    #print(record[r])
    # fitting the model
    timesteps = tunning_dict[t]['timesteps']
    epochs = tunning_dict[t]['epochs']
    batch_size= tunning_dict[t]['batch_size']
    multisteps= tunning_dict[t]['multisteps']
    cv_splits = tunning_dict[t]['cv_splits']
    #print(timesteps,epochs,batch_size,cv_splits)

    X, y = to_supervised(df, timesteps)

    model = build_model(timesteps, n_variate)
    model, history = compile_and_fit(model, epochs, batch_size)
    #print("df: ",df.shape," timesteps",timesteps," multisteps ",multisteps)
    forecasts = forecast(model, df, timesteps, multisteps, scaler)

    print(forecasts)

    prev = []

    #plot do valor previsto da ação de Open
    for f in forecasts:
        prev.append(f[0][0])

    print('Previsões:')
    print(prev)
    plot_forecast(df_raw, prev)

    #Scorer
  

    record[t] = history

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
Epoch 1/25
2/2 [==============================] - 4s 110ms/step - loss: 0.3005 - mae: 0.2314 - rmse: 0.3005
Epoch 2/25
2/2 [==============================] - 0s 59ms/step - loss: 0.2735 - mae: 0.2199 - rmse: 0.2735
Epoch 3/25
2/2 [==============================] - 0s 50ms/step - loss: 0.2490 - mae: 0.2083 - rmse: 0.2490
Epoch 4/25
2/2 [==============================] - 0s 60ms/step - loss: 0.2270 - mae: 0.1980 - rmse: 0.2270
Epoch 5/25
2/2 [==============================] - 0s 50ms/step - loss: 0.2106 - mae: 0.1976 - rmse: 0.2106
Epoch 6/25
2/2 [==============================] - 0s 58ms/step - loss: 0.2042 - mae: 0.1968 - rmse: 0.2042
Epoch 7/25
2/2 [==============================] - 0s 64ms/step - loss: 0.2042 - mae: 0.1964 - rmse: 0.2042
Epoch 8/25
2/2 [==============================] - 0s 68ms/step - loss: 0.2049 - mae: 0.1995 - rmse: 0.

KeyError: 'val_loss'

In [ ]:
id_tunning = 1
id_split =1

final_dict = {}

for r in record:
#print(tunning_dict[1]['epochs'])
    loss = []
    mae =[]
    rmse = []
    val_loss = []
    val_mae = []
    val_rmse = []

    for h in record[r]:
        #print("Tunning ID:  ",id_tunning," Split ID: ",id_split)
        #plot_learning_curves(h, tunning_dict[id_tunning]['epochs'])
        #['loss', 'mae', 'rmse', 'val_loss', 'val_mae', 'val_rmse']
        #print("loss: ",sum(h.history['loss'])/len(h.history['loss'])," MAE: ",sum(h.history['mae'])/len(h.history['mae'])," RMSE: ",sum(h.history['rmse'])/len(h.history['rmse'])," VAL_LOSS: ",sum(h.history['val_loss'])/len(h.history['val_loss'])," VAL_MAE: ",sum(h.history['val_mae'])/len(h.history['val_mae'])," VAL_RMSE: ",sum(h.history['val_rmse'])/len(h.history['val_rmse']))
        loss.append(sum(h.history['loss'])/len(h.history['loss']))
        mae.append(sum(h.history['mae'])/len(h.history['mae']))
        rmse.append(sum(h.history['rmse'])/len(h.history['rmse']))
        val_loss.append(sum(h.history['val_loss'])/len(h.history['val_loss']))
        val_mae.append(sum(h.history['val_mae'])/len(h.history['val_mae']))
        val_rmse.append(sum(h.history['val_rmse'])/len(h.history['val_rmse']))
        id_split+=1
    id_split=1
  
    final_dict[id_tunning]=[sum(loss)/len(loss), sum(mae)/len(mae),sum(rmse)/len(rmse),sum(val_loss)/len(val_loss),sum(val_mae)/len(val_mae), sum(val_rmse)/len(val_rmse)]

    id_tunning=id_tunning+1



In [ ]:
for f in final_dict:
    print("Loss | MAE | RMSE | VAL_LOSS | VAL_MAE | VAL_RMSE")
    print("ID tunning: ",f, " Valores: ",final_dict[f],"\n")

final_df = pd.DataFrame.from_dict(final_dict, orient='index')
final_df.columns = ['Loss','MAE','RMSE','VAL_LOSS','VAL_MAE','VAL_RMSE']
final_df

In [ ]:
#Results metrics to a file
final_df.to_csv(r"resultados_lstm_covid.csv",index=True)